# Saturation - Maximum Find

In [19]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Algoritmi

### TODO: Building Tree



### Saturation (Plug-in Full-Saturation)

In [42]:
# -*- coding: utf-8 -*-
from pymote.algorithm import NodeAlgorithm
from pymote.message import Message

class Saturation(NodeAlgorithm): 
    
    #required_params = ('informationKey', 'treeKey',) # must have ','
    required_params = ('informationKey',) # must have ','
    default_params = {'treeNeighbors': 'Neighbors','parentKey' : 'Parent', 'treeKey': 'Tree'}

    #TODO: treeKey now serves as treeNeighbors, and treeNeighbors as treeKey - necessary to SWAP them
    #      should treeKey be in required_params?!
    
    def initializer(self):
        ini_nodes = []
        for node in self.network.nodes():
            node.memory[self.treeNeighbors] = node.compositeSensor.read()['Neighbors']
            node.memory[self.treeKey]= list(node.memory[self.treeNeighbors])
            self.initialize(node)
            node.status = 'AVAILABLE'
            if self.informationKey in node.memory:
                node.status = 'AVAILABLE'
                ini_nodes.append(node)
        for ini_node in ini_nodes:
            self.network.outbox.insert(0, Message(header=NodeAlgorithm.INI,destination=ini_node))  # to je spontani impuls   

        
    def available(self, node, message):

        #inicijatori
        if message.header == NodeAlgorithm.INI: #Spontaneously
            node.send(Message(header='Activate', data='Activate'))
            #initialize() mislim da ipak treba biti u initalizeru
            if len(node.memory[self.treeNeighbors])==1 : #ako je čvor list
                node.memory[self.parentKey] = list(node.memory[self.treeNeighbors])
                updated_data=self.prepare_message(node)
                node.send(Message(header='M', data = updated_data, destination = node.memory[self.parentKey]))
                node.status = 'PROCESSING'
            else:
                node.status = 'ACTIVE' #izvrši se
                
        if message.header == 'Activate':
            destination_nodes = list(node.memory[self.treeNeighbors])
            print "message.source:"
            print type(message.source)
            print message.source
            
            node.send(Message(header='Activate', data='Activate', destination=destination_nodes.remove(message.source)))
            print destination_nodes
            #initialize() mislim da ipak treba biti u initalizeru            
            if len(node.memory[self.treeNeighbors])==1 :
                node.memory[self.parentKey] = list(node.memory[self.treeNeighbors])                
                updated_data=self.prepare_message(node)
                node.send(Message(header='M', data=updated_data, destination=node.memory[self.parentKey]))
                #dal prepare?
                node.status = 'PROCESSING'
            else:
                node.status='ACTIVE' #izvrši se
    
    def active(self, node, message):  
        

        if message.header=='M':
            self.process_message(node,message)
            ###ključno
            node.memory[self.treeNeighbors].remove(message.source) # ne radi?

            ##izgleda da neki ostanu bez susjeda kao posljedica ovoga, ne, trebali bi uci u processing jopš u availabele
            
            if len(node.memory[self.treeNeighbors])==1 :
                node.memory[self.parentKey] = list(node.memory[self.treeNeighbors])                
                updated_data=self.prepare_message(node)
                node.send(Message(header='M', data=updated_data, destination=node.memory[self.parentKey]))
                node.status = 'PROCESSING'

    def processing(self, node, message):
        if message.header=="M":           
            self.process_message(node,message)
            #self.resolve(node)
            node.staus='SATURATED'
        #if message.header=="Notification":
            #print "Nebi smio biti tu"
            
    def prepare_message(self,node):
        raise NotImplementedError
        
    def process_message(self, node, message):
        raise NotImplementedError

    def initialize(self, node):
        raise NotImplementedError
                
    def resolve(self,node):
        raise NotImplementedError
        
    STATUS = {
              'AVAILABLE': available,
              'ACTIVE': active,
              'PROCESSING': processing,
              'SATURATED': resolve,
             }

### MaxFind

In [43]:
# -*- coding: utf-8 -*-
from pymote.sensor import TempSensor

class MaxFind(Saturation):
    #required_params = ('dataKey',) 
    default_params = {'temperatureKey':'Temperature','maxKey':'Max'}

    def processing(self,node,message):
        if message.header=="M":
            self.process_message(node,message)
            #node.status = 'SATURATED'
            self.resolve(node)
            
        if message.header=="Notification":
            destination_nodes = node.memory[self.treeKey]
            #print destination_nodes
            #print node.memory[self.parentKey]
            #print type(node.memory[self.parentKey])
            #print node.memory[self.parentKey][0]
            
            self.process_message(node,message)
            destination_nodes.remove(node.memory[self.parentKey][0])            
            
            node.send(Message(header='Notification', data=node.memory[self.maxKey], destination=destination_nodes))
            
            if node.memory[self.temperatureKey]==message.data:
                node.status="MAXIMUM"
            else:
                node.status="SMALLER"
    
    def initialize(self, node):
        node.compositeSensor=(TempSensor,'Temperature')
        node.memory[self.temperatureKey]=node.compositeSensor.read()['Temperature']
        node.memory[self.maxKey]=node.memory[self.temperatureKey]   
        #return node.memory[self.maxKey] is not None    
    
    def prepare_message(self,node):
        return node.memory[self.maxKey]
                   
    def process_message(self,node,message):
        #print message.data
        if message.data>node.memory[self.maxKey]:
            node.memory[self.maxKey] = message.data
    
    def resolve(self,node):
        #print "TU SAM"
        destination_nodes = node.memory[self.treeKey]  #garantira topologiju      
        #print destination_nodes
        #print node.memory[self.parentKey]
        destination_nodes.remove(node.memory[self.parentKey][0]) #garantira topologiju        
        
        node.send(Message(header='Notification', data=node.memory[self.maxKey], destination=destination_nodes))        

        if node.memory[self.temperatureKey] == node.memory[self.maxKey]:
            node.status='MAXIMUM'
        else :
            node.status='SMALLER'
    
                                                 
    def mini(self,node,message):
        print "MAXIMUM"
    def larger(self,node,message):
        print "SMALLER"
    
    STATUS = {
              'MAXIMUM' : mini,
              'SMALLER' : larger,
              'AVAILABLE': Saturation.STATUS.get('AVAILABLE'),
              'ACTIVE': Saturation.STATUS.get('ACTIVE'),
              #'PROCESSING':Saturation.STATUS.get('PROCESSING'), #redefinirali smo processing
              'PROCESSING':processing, #redefinirali smo processing
              #'SATURATED':Saturation.STATUS.get('SATURATED'),
              'SATURATED':resolve,
        
        #svaka metoda vezana uz status mora imati self,node,message
        #procedure primaju self po defaultu kao prvi argument, 
        #ako je procedura definirana kao def procedura(self,node) 
        #krivo je zvati self.procedura(self,node), 
        #ispravno je     self.procedura(node)
             }    

### Random Mreža

In [44]:
from pymote.networkgenerator import NetworkGenerator
from pymote.npickle import write_pickle, read_pickle
#net_gen = NetworkGenerator(3)
net_gen = NetworkGenerator(5)
#net_gen = NetworkGenerator(15)
#net_gen = NetworkGenerator(500)
net = net_gen.generate_random_network()

In [ ]:
net.show() #po gašenju izaziva Dead Kernel

### Mreža stablo

In [45]:
from pymote.network import Network
net = Network()
root=net.add_node(pos=[300,500])
root.commRange=150
node=net.add_node(pos=[200,400])
node.commRange=150
node=net.add_node(pos=[400,400])
node.commRange=150

node=net.add_node(pos=[100,300])
node.commRange=150

node=net.add_node(pos=[250,300])
node.commRange=150
node=net.add_node(pos=[500,300])
node.commRange=150
node=net.add_node(pos=[450,200])
node.commRange=150
node=net.add_node(pos=[10,200])
node.commRange=150

### Mreža Lanac (n = 4, m = 5)

In [ ]:
from pymote.network import Network
net = Network()
n = 5
u = 100

for i in range(0,n*u,u):
    node = net.add_node(pos=[1,i])
    node.commRange=110

### Mreža Zvijezda (n = 4, m = 5)

In [ ]:
from pymote.network import Network
net = Network()
n = 5
for i in range(100,(n+1)*100,100):
    node= net.add_node(pos=[i,i])
    node.commRange=200

node= net.add_node(pos=[100,300])
node.commRange=200
node= net.add_node(pos=[300,100])
node.commRange=200

### Simulacija

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from pymote.algorithms.saturation import Saturation
from pymote.algorithms.maximumfind import MaxFind

In [46]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from pymote.npickle import write_pickle, read_pickle
from pymote.simulation import Simulation

#net = read_pickle('mrezaRandom.tar.gz') 
#net = read_pickle('mrezaList.tar.gz')
#net = read_pickle('mrezaZvijezda.tar.gz')
#net = read_pickle('mrezaStablo.tar.gz')

from pymote.network import Network
net = Network()
root=net.add_node(pos=[300,500])
root.commRange=150
node=net.add_node(pos=[200,400])
node.commRange=150
node=net.add_node(pos=[400,400])
node.commRange=150
node=net.add_node(pos=[100,300])
node.commRange=150
node=net.add_node(pos=[250,300])
node.commRange=150
node=net.add_node(pos=[500,300])
node.commRange=150
node=net.add_node(pos=[450,200])
node.commRange=150
node=net.add_node(pos=[10,200])
node.commRange=150

net.algorithms = ( (MaxFind,{'informationKey':'I'}), )
some_node = net.nodes()[4]# uzimamo prvi cvor u listi cvorova mreze
some_node.memory['I'] = 'Activate'

#some_node_2 = net.nodes()[1]# uzimamo prvi cvor u listi cvorova mreze
#some_node_2.memory['I'] = 'Activate'


write_pickle(net,'mrezaStabloAfter.tar.gz')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
sim = Simulation(net)
sim.run()


#write_pickle(net,'mrezaStabloAfter.tar.gz')
#write_pickle(net,'mrezaStabloAfter.txt')

suma=0

print "Number of nodes:"
print len(list(net.nodes()))
print "Number of edges:"
print len(list(net.edges()))
print "Total number of messages:"
print suma

for node in net.nodes():
    #print node.memory['Max']
    print node.id
    print node.memory
    print node.status
    #suma=suma+node.memory['msgKey']

DEBUG   [simulation.py]: Simulation 0x10f490f8 created successfully.


DEBUG:pymote.simulation:Simulation 0x10f490f8 created successfully.


DEBUG   [simulation.py]: [MaxFind] Step 1 finished


DEBUG:pymote.simulation:[MaxFind] Step 1 finished


DEBUG   [simulation.py]: [MaxFind] Step 2 finished


...

DEBUG   [simulation.py]: [MaxFind] Algorithm finished


DEBUG:pymote.simulation:[MaxFind] Algorithm finished


INFO    [simulation.py]: Simulation has finished. There are no algorithms left to run. To run it from the start use sim.reset().


INFO:pymote.simulation:Simulation has finished. There are no algorithms left to run. To run it from the start use sim.reset().


Number of nodes:
8
Number of edges:
7
Total number of messages:
0
139
{'Neighbors': [<Node id=141>], 'Max': 37, 'Tree': [<Node id=140>], 'Temperature': 30, 'Parent': [<Node id=141>]}
SMALLER
140
{'Neighbors': [<Node id=139>], 'Max': 37, 'Tree': [<Node id=142>, <Node id=143>], 'Temperature': 37, 'Parent': [<Node id=139>]}
MAXIMUM
141
{'Neighbors': [<Node id=139>], 'Max': 37, 'Tree': [<Node id=144>], 'Temperature': 2, 'Parent': [<Node id=139>]}
SMALLER
142
{'Neighbors': [<Node id=140>], 'Max': 37, 'Tree': [<Node id=146>], 'Temperature': 31, 'Parent': [<Node id=140>]}
SMALLER
143
{'Neighbors': [<Node id=140>], 'Temperature': 3, 'Parent': [<Node id=140>], 'I': 'Activate', 'Max': 37, 'Tree': []}
SMALLER
144
{'Neighbors': [<Node id=141>], 'Max': 37, 'Tree': [<Node id=145>], 'Temperature': 20, 'Parent': [<Node id=141>]}
SMALLER
145
{'Neighbors': [<Node id=144>], 'Max': 37, 'Tree': [], 'Temperature': 35, 'Parent': [<Node id=144>]}
SMALLER
146
{'Neighbors': [<Node id=142>], 'Max': 37, 'Tree': [

In [ ]:
net.show()

In [ ]:
print node.memory
#list(node.memory['Neighbors']).remove(node.memory['Parent'])

type(node.memory['Neighbors'])
node.memory['Neighbors'].remove(node.memory['Neighbors'][0])
print node.memory

In [ ]:
print node.memory